# langchain 学习

## 提示词仓库
可以通过hub直接来获取社区的提示词。

In [2]:
# set the LANGCHAIN_API_KEY environment variable (create key in settings)
from langchain import hub
prompt = hub.pull("hardkothari/prompt-maker")
prompt


ChatPromptTemplate(input_variables=['lazy_prompt', 'task'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert Prompt Writer for Large Language Models.\n\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lazy_prompt', 'task'], template='Your goal is to improve the prompt given below for {task} :\n--------------------\n\nPrompt: {lazy_prompt}\n\n--------------------\n\nHere are several tips on writing great prompts:\n\n-------\n\nStart the prompt by stating that it is an expert in the subject.\n\nPut instructions at the beginning of the prompt and use ### or to separate the instruction and context \n\nBe specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc \n\n---------\n\nHere\'s an example of a great prompt:\n\nAs a master YouTube content creator, develop an engaging script that revolves around the theme of "Exploring Ancient Ruins."\n\nYour scrip

## 接入llm

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi3.5")
llm.invoke("Tell me a joke")

"Sure, here'mith: Why don't scientists trust atoms? Because they make up everything, even in jokes! Now isn’t that elemental humor. (Note to self and future users alike—scientific references are always welcome when weaving them into a jest!)"

## 调用Agent

### 集成的tools列表
https://python.langchain.com/v0.2/docs/integrations/tools/

In [15]:
%pip install -qU duckduckgo-search


Note: you may need to restart the kernel to use updated packages.


In [45]:
# 用代理实现访问duckduckgo
from langchain.tools import DuckDuckGoSearchRun
from langchain.callbacks.manager import CallbackManagerForToolRun
from typing import Optional, List, Dict
from langchain.utilities import DuckDuckGoSearchAPIWrapper
    
class CustomDuckDuckGoSearchAPIWrapper(DuckDuckGoSearchAPIWrapper):
    proxy: Optional[str] = None  # Add a proxy attribute

    def _ddgs_text(
        self, query: str, max_results: Optional[int] = None
    ) -> List[Dict[str, str]]:
        """Run query through DuckDuckGo text search and return results."""
        from duckduckgo_search import DDGS

        with DDGS(proxy=self.proxy) as ddgs:
            ddgs_gen = ddgs.text(
                query,
                region=self.region,
                safesearch=self.safesearch,
                timelimit=self.time,
                max_results=max_results or self.max_results,
                backend=self.backend,
            )
            if ddgs_gen:
                return [r for r in ddgs_gen]
        return []
    
wrapper = CustomDuckDuckGoSearchAPIWrapper(proxy="http://127.0.0.1:7890")
custom_search_tool = DuckDuckGoSearchRun(api_wrapper=wrapper)

In [51]:
from langchain import hub
from langchain_community.llms import Ollama
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")
llm = Ollama(model="mistral-small:latest")
tools = [custom_search_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke({"input": "今天北京气温有多少度？请用摄氏度来回答。"})



> Entering new AgentExecutor chain...
 Action: duckduckgo_search
Action Input: "Beijing weather today in Celsius"Beijing, China - Current temperature and weather conditions. Detailed hourly weather forecast for today - including weather conditions, temperature, pressure, humidity, precipitation, dewpoint, wind, visibility, and UV index data. Updated 10/17 6:47 PM. The hourly local weather forecast shows hour by hour weather conditions like temperature, feels like temperature, humidity, amount of precipitation and chance of precipitation, wind and gusts for Beijing. Beijing. Beijing Weather Forecast. Providing a local hourly Beijing weather forecast of rain, sun, wind, humidity and temperature. The Long-range 12 day forecast also includes detail for Beijing weather today. Live weather reports from Beijing weather stations and weather warnings that include risk of thunder, high UV index and forecast gales. Beijing Weather Forecasts. Weather Underground provides local & long-range weath

{'input': '今天北京气温有多少度？请用摄氏度来回答。', 'output': '今天北京的气温是10摄氏度。'}